In [ ]:
%gui qt

from OCC.Display.SimpleGui import init_display
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Pln, gp_Vec
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_EDGE, TopAbs_FACE, TopAbs_VERTEX
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface, BRepAdaptor_Curve
from OCC.Core.GeomAbs import GeomAbs_Plane, GeomAbs_Cylinder

from utils_geometry import *
from utils_dstv import *
from utils_visualization import *
from utils_classification import *
from utils_reports import *

import numpy as np
import json

DSTV_FACE_MAP = {'I':['O','U','V'], 'U':['H','U','O'], 'L':['H','U']}

# Start display without blocking
display, start_display, add_menu, add_function_to_menu = init_display()

display.EraseAll()
display.View_Iso()
display.FitAll()

In [ ]:
# Load the STEP and extract 'solid'
from OCC.Extend.DataExchange import read_step_file
# shape = read_step_file("../data/0444-1 ANGLED.step")
# shape = read_step_file("../data/ncTest.step")
# shape = read_step_file("../data/TestEA.step")
# shape = read_step_file("../data/TestEAMirror.step")
# shape = read_step_file("../data/TestUEA.step")
# shape = read_step_file("../data/TestUEAMirror.step")
shape = read_step_file("../data/TestPFC.step")
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID
exp = TopExp_Explorer(shape, TopAbs_SOLID)
solid = exp.Current()

In [ ]:

def fingerprint_shape(solid, obb_dims):
    """
    Analyze face normals and areas to guess profile type: Beam, Channel, Angle, Unknown
    """
    face_areas = []
    face_normals = []

    exp = TopExp_Explorer(solid, TopAbs_FACE)
    while exp.More():
        face = topods.Face(exp.Current())
        surf = BRepAdaptor_Surface(face)
        if surf.GetType() != GeomAbs_Plane:
            exp.Next()
            continue

        # Normal
        n = surf.Plane().Axis().Direction()
        n = np.array([n.X(), n.Y(), n.Z()], dtype=float)
        n /= np.linalg.norm(n)

        # Area
        gp = GProp_GProps()
        brepgprop.SurfaceProperties(face, gp)
        area = gp.Mass()

        face_normals.append(n)
        face_areas.append(area)
        exp.Next()

    # Sort OBB dimensions
    dims = sorted(obb_dims)  # [flange_thickness, web_depth, length]
    thickness, height, length = dims
    slenderness = length / height if height else 0
    aspect_ratio = height / thickness if thickness else 0

    # Area stats
    largest = max(face_areas) if face_areas else 0
    large_faces = [a for a in face_areas if a > 0.8 * largest]

    print("\n🔎 Shape fingerprint debug:")
    print(f"  OBB dimensions (sorted): {np.round(dims,1)}")
    print(f"  Slenderness ratio (L/H): {slenderness:.2f}")
    print(f"  Aspect ratio (H/t):      {aspect_ratio:.2f}")
    print(f"  Total planar faces:      {len(face_areas)}")
    print(f"  Large planar faces:      {len(large_faces)}")
    print(f"  Max area:                {max(face_areas):.1f}" if face_areas else "  Max area: —")
    print(f"  Large face areas:        {[round(a,1) for a in large_faces]}")
    
    # Fingerprint logic
    
    # --- Check for Beam ---
    if len(face_areas) >= 10 and slenderness > 4 and max(face_areas) > 100000:
        return "I"
    
    # --- Check for Angles ---
    elif len(large_faces) == 2 and slenderness > 6:
        a1, a2 = large_faces
        area_ratio = max(a1, a2) / min(a1, a2)
        if area_ratio < 1.1:
            return "L"
        else:
            return "L"

    # --- Check for Channel ---
    elif len(face_areas) >= 6 and slenderness > 6 and max(face_areas) > 50000:
        return "U"
        
    # --- Fallback ---
    return "Unknown"


In [ ]:
def classify_profile(cs, json_path, tol_dim=1.0, tol_area=0.05):
    """
    Attempts to classify a structural profile by matching its dimensions and area.
    Tries both the original and swapped width/height to handle OBB axis flips.
    """
    with open(json_path) as f:
        lib = json.load(f)

    def try_match(height, width):
        best, best_score = None, float('inf')
        for cat, ents in lib.items():
            for name, info in ents.items():
                dh = abs(height - info["height"])
                dw = abs(width  - info["width"])
                if dh > tol_dim or dw > tol_dim:
                    continue
                ea = abs(cs["area"] - info["csa"]) / info["csa"]
                if ea > tol_area:
                    continue
                el = abs(cs["length"] - info.get("length", cs["length"])) / info.get("length", cs["length"])
                score = dh + dw + ea * 100 + el * 100
                if score < best_score:
                    best_score = score
                    best = {
                        **info,
                        "Designation": name,
                        "Category": cat,
                        "Measured_height": height,
                        "Measured_width": width,
                        "Measured_area": cs["area"],
                        "Measured_length": cs["length"],
                        "Match_score": score,
                        "Profile_type": cs.get("profile_type", "Unknown")
                    }
        return best, best_score

    # Try both normal and swapped width/height
    original, score1 = try_match(cs["span_web"], cs["span_flange"])
    swapped, score2  = try_match(cs["span_flange"], cs["span_web"])

    if original and (not swapped or score1 <= score2):
        return original
    elif swapped:
        swapped["Swapped_dimensions"] = True
        return swapped
    else:
        return None